In [22]:
from sqlalchemy import create_engine
import pandas as pd
import os
import sqlite3

pd.set_option("display.max_rows", 100)

con = sqlite3.connect("olist.db")
cur = con.cursor()

engine = create_engine('sqlite:///olist.db', echo=False)

In [11]:
cur.execute("""create table customer(
    customer_id TEXT NOT NULL, 
    customer_unique_id TEXT NOT NULL, 
    customer_zip_code_prefix INTEGER NOT NULL, 
    customer_city TEXT NOT NULL, 
    customer_state TEXT NOT NULL
)""")

cur.execute("""create table geolocation(
    geolocation_zip_code_prefix TEXT NOT NULL, 
    geolocation_lat TEXT NOT NULL, 
    geolocation_lng INTEGER NOT NULL, 
    geolocation_city TEXT NOT NULL, 
    geolocation_state TEXT NOT NULL
)""")

cur.execute("""create table item(
    order_id TEXT NOT NULL, 
    order_item_id TEXT NOT NULL, 
    product_id INTEGER NOT NULL, 
    seller_id TEXT NOT NULL, 
    shipping_limit_date TEXT NOT NULL,
    price REAL NOT NULL,
    freight_value REAL NOT NULL
)""")

cur.execute("""create table payments(
    order_id TEXT NOT NULL, 
    payment_sequential INTEGER NOT NULL, 
    payment_type TEXT NOT NULL, 
    payment_installments INTEGER NOT NULL, 
    payment_value REAL NOT NULL
)""")

cur.execute("""
create table reviews(
    review_id TEXT NOT NULL, 
    order_id TEXT NOT NULL, 
    review_score INTEGER NOT NULL, 
    review_comment_title TEXT, 
    review_comment_message TEXT,
    review_creation_date TEXT NOT NULL,
    review_answer_timestamp TEXT NOT NULL
)""")

cur.execute("""
create table orders(
    order_id,
    customer_id TEXT NOT NULL,  
    order_status TEXT NOT NULL,
    order_purchase_timestamp TEXT NOT NULL,
    order_approved_at TEXT,
    order_delivered_carrier_date TEXT,
    order_delivered_customer_date TEXT,
    order_estimated_delivery_date TEXT NOT NULL
)
""")

cur.execute("""
create table products(
    product_id TEXT NOT NULL,
    product_category_name TEXT NOTN ULL,
    product_name_lenght TEXT NOT NULL, 
    product_description_lenght INTEGER NOT NULL, 
    product_photos_qty INTEGER NOT NULL, 
    product_weight_g INTEGER NOT NULL, 
    product_lenght_cm INTEGER NOT NULL, 
    product_height_cm INTEGER NOT NULL, 
    product_with_cm INTEGER NOT NULL
)
""")

cur.execute("""
create table sellers(
    seller_id TEXT NOT NULL,
    seller_zip_code_prefix TEXT NOT NULL,
    seller_city TEXT NOT NULL,
    seller_state TEXT NOT NULL
)
""")

In [12]:
customer = pd.read_csv("./archive/olist_customers_dataset.csv")
items = pd.read_csv("./archive/olist_order_items_dataset.csv")
payments = pd.read_csv("./archive/olist_order_payments_dataset.csv")
geolocation = pd.read_csv("./archive/olist_geolocation_dataset.csv")
order = pd.read_csv("./archive/olist_orders_dataset.csv")
sellers = pd.read_csv("./archive/olist_sellers_dataset.csv")
products = pd.read_csv("./archive/olist_products_dataset.csv")
review = pd.read_csv("./archive/olist_order_reviews_dataset.csv")

In [19]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


In [23]:
products["product_category_name"].value_counts()

product_category_name
cama_mesa_banho                                   3029
esporte_lazer                                     2867
moveis_decoracao                                  2657
beleza_saude                                      2444
utilidades_domesticas                             2335
automotivo                                        1900
informatica_acessorios                            1639
brinquedos                                        1411
relogios_presentes                                1329
telefonia                                         1134
bebes                                              919
perfumaria                                         868
papelaria                                          849
fashion_bolsas_e_acessorios                        849
cool_stuff                                         789
ferramentas_jardim                                 753
pet_shop                                           719
eletronicos                                

In [ ]:
def category_adjust(df):
    df.replace(["cama_mesa_banho", "casa_conforto"], "cama, mesa, banho", inplace=True)
    df.replace(["moveis_decoracao", "moveis_escritorio", "moveis_sala", "moveis_cozinha_area_de_servico_jantar_e_jardim", "moveis_quarto", "moveis_colchao_e_estofado"], "móveis e decoração", inplace=True)